In [17]:
import numpy as np
from scipy.integrate import solve_ivp
from SALib.sample import saltelli
from SALib.analyze import sobol
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
def model(t, z, alpha, K, beta1, beta2, C1star, 
          alpha1, K1, K2, l1, l2, gamma1, gamma2, b1, b2, mu1):
    
    C1 = (alpha1/mu1)*z[1]
    f_C1 = beta2*C1/(C1star + C1)
    Ti = z[0] + z[1] + z[2]

    dTu = alpha*z[0]*(1 - (Ti/K)) - beta1*(z[0]*z[3]/(K1 + Ti)) - f_C1*z[0]*z[4]/(K2 + Ti)
    dT1 = beta1*(z[0]*z[3]/(K1 + Ti)) - l1*z[1]
    dT2 = f_C1*z[0]*z[4]/(K2 + Ti) - l2*z[2]
    dV1 = b1*l1*z[1] - beta1*(z[0]*z[3]/(K1 + Ti)) - gamma1*z[3]
    dV2 = b2*l2*z[2] - f_C1*z[0]*z[4]/(K2 + Ti) - gamma2*z[4]

    dz = [dTu, dT1, dT2, dV1, dV2]

    return dz

In [19]:
z0 = [1e4, 0, 0, 1e3, 1e3]
t = np.linspace(0, 200, 1000)
t_span = [0, 200]

alpha = 0.1195
K = 7.2e9
beta1 = 0.2
beta2 = 0.01
K1 = 1e5
K2 = 1e5
l1 = 0.001
l2 = 0.04
gamma1 = 0.02
gamma2 = 0.02
b1 = 50
b2 = 500
mu1 = 0.02
alpha1 = 1
C1star = 1e3

z = solve_ivp(model, t_span, z0, method='Radau', t_eval=t, args = (alpha, K, beta1, beta2, C1star, 
          alpha1, K1, K2, l1, l2, gamma1, gamma2, b1, b2, mu1))


In [20]:
problem = {
    'num_vars': 15,
    'names': ['alpha', 'K', 'beta1', 'beta2', 'C1star', 
          'alpha1', 'K1', 'K2', 'l1', 'l2', 'gamma1', 'gamma2', 'b1', 'b2', 'mu1'],
    'bounds': [[0.1, 0.7],
               [1e6, 1e9],
               [0.1, 0.5],
               [0.001, 0.1],
               [1e4, 1e5],
               [1e4, 1e5],
               [0.001, 0.1],
               [0.01, 0.1],
               [0.001, 0.1],
               [0.001, 0.1],
               [1, 500],
               [50, 8000],
               [0.001, 0.1],
               [0.1, 10],
               [10, 1e4]]
}

vals = saltelli.sample(problem, 64)
Y = np.zeros((len(vals), len(t)))

for i in range(len(vals)):
    params = vals[i]
    alpha = params[0]
    K = params[1]
    beta1 = params[2]
    beta2 = params[3]
    C1star = params[4]
    alpha1 = params[5]
    K1 = params[6]
    K2 = params[7]
    l1 = params[8]
    l2 = params[9]
    gamma1 = params[10]
    gamma2 = params[11]
    b1 = params[12]
    b2 = params[13]
    mu1 = params[14]

    z = solve_ivp(model, t_span, z0, method='Radau', t_eval=t, args = (alpha, K, beta1, beta2, C1star, 
          alpha1, K1, K2, l1, l2, gamma1, gamma2, b1, b2, mu1))
    
    Tu = z.y[0]
    
    Y[i] = Tu 


In [25]:
print(len(vals))

2048


In [21]:
Indicies = sobol.analyze(problem, Y, print_to_console=True)

ValueError: could not broadcast input array from shape (64,1000) into shape (64000,)